<a href="https://colab.research.google.com/github/mrheavyman/Price_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [11]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [13]:
import pandas as pd
from typing import List

URL = "https://www.iban.com/country-codes"

def _flatten_columns(cols: pd.Index) -> List[str]:
  if isinstance(cols, pd.MultiIndex):
    return [
        " ".join([str(part) for part in tup if part is not None and str(part) != "nan"]).strip()
        for tup in cols.tolist()
    ]
  else:
    return [str(c).strip() for c in cols.tolist()]
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    tables = pd.read_html(html)
    if not tables:
      raise ValueError("No tables found in provided HTML")
    for t in tables:
      if t.shape[1] >= 3:
        t = t.copy()
        t.columns = _flatten_columns(t.columns)
        t = t.loc[:, [c for c in t.columns if c and c.strip()]]
        return t

resp = requests.get(URL, timeout=30)
resp.raise_for_status()
html = resp.text

df_raw = q1_read_table(html)

assert 'df_raw' in globals(), "df_raw not found in previous step."

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    df = df.copy()
    col_map = {}

    for c in df.columns:
      c_norm = c.strip().lower()
      if c_norm in ("country", "country/territory", "country or area"):
        col_map[c] = "Country"
      elif "alpha-2" in c_norm or "alpha 2" in c_norm:
        col_map[c] = "Alpha-2"
      elif "alpha-3" in c_norm or "alpha 3" in c_norm:
        col_map[c] = "Alpha-3"
      elif "numeric" in c_norm or "num" in c_norm or "iso 3166-1 numeric" in c_norm:
        col_map[c] = "Numeric"
      else:
        col_map[c] = c.strip()
    df.rename(columns=col_map, inplace=True)

    # Fix typo: df.coulmns to df.columns
    for c in df.columns:
      if pd.api.types.is_object_dtype(df[c]):
        df[c] = df[c].astype(str).str.strip()

    if "Alpha-2" in df.columns:
      df["Alpha-2"] = df["Alpha-2"].str.upper()
    if "Alpha-3" in df.columns:
      df["Alpha-3"] = df["Alpha-3"].str.upper()

    if "Numeric" in df.columns:
      df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

    required_cols_to_check = [c for c in ["Country", "Alpha-2", "Alpha-3", "Numeric"] if c in df.columns]

    if required_cols_to_check:
      df = df.dropna(subset=required_cols_to_check)
      for c in required_cols_to_check:
        if pd.api.types.is_object_dtype(df[c]):
          df = df[df[c].astype(str).str.strip() != ""]

    # Select only the explicitly requested columns
    requested_final_cols = [c for c in ["Country", "Alpha-2", "Alpha-3", "Numeric"] if c in df.columns]

    return df[requested_final_cols]

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by Numeric desc and return Top-N. TODO: implement."""
    df = df.copy()
    if "Numeric" in df.columns:
      if "Country" in df.columns:
        df = df.sort_values(by=["Numeric", "Country"], ascending=[False, True])
      else:
        df = df.sort_values(by=["Numeric"], ascending=[False])
    return df.head(top).reset_index(drop=True)

    readme_text = """# Assignment 6, Question 1 — Web Scraping

## URL
https://www.iban.com/country-codes

## Steps
1.  **Fetch HTML**: The HTML content from the specified URL was fetched using `requests`.
2.  **Extract Table**: The `q1_read_table` function used `pandas.read_html` to identify and extract the main country codes table. It then flattened the multi-level headers to single-level headers for easier processing.
3.  **Clean Data**: The `q1_clean` function performed the following cleaning steps:
    *   Standardized column names to `Country`, `Alpha-2`, `Alpha-3`, and `Numeric`.
    *   Trimmed whitespace from all string columns.
    *   Converted `Alpha-2` and `Alpha-3` codes to uppercase.
    *   Converted the `Numeric` column to a nullable integer type, coercing non-numeric values to `NaN`.
    *   Dropped rows where `Country`, `Alpha-2`, `Alpha-3`, or `Numeric` had missing values (after conversion).
4.  **Save Output**: The cleaned DataFrame will be saved to `data_q1.csv`.
5.  **Print Top-15**: The `q1_sort_top` function sorts the cleaned DataFrame by `Numeric` in descending order (and then by `Country` alphabetically for ties) and returns the top 15 entries, which will be printed to the console.

## Limitation
The `pandas.read_html` function is highly dependent on the HTML structure of the target page. If the website's layout changes significantly, especially the table's structure or its surrounding tags, the `q1_read_table` function might fail to correctly identify or parse the desired table, requiring updates to the parsing logic.
"""






/tmp/ipython-input-1163708185.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import re

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    items = []
    for row in soup.select("tr.athing"):

        rank_text = row.select_one("span.rank")
        rank = rank_text.get_text(strip=True).split(".")[0] if rank_text else ""

        titlelink = row.select_one("span.titleline a")
        title = titlelink.get_text(strip=True) if titlelink else ""
        link = titlelink.get("href", "") if titlelink else ""


        sub = row.find_next_sibling("tr")
        subtext = sub.select_one(".subtext") if sub else None


        points_el = subtext.select_one("span.score") if subtext else None
        points = points_el.get_text(strip=True) if points_el else ""


        user_el = subtext.select_one("a.hnuser") if subtext else None
        user = user_el.get_text(strip=True) if user_el else ""

        comments = ""
        if subtext:
            links = subtext.select("a")
            if links:
                comments = links[-1].get_text(strip=True)

        items.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })
    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")
